In [5]:
import pandas as pd
from sqlalchemy import create_engine, types

In [2]:
from collections import defaultdict
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Autenticación
auth_manager = SpotifyClientCredentials(client_id="ce08bdf0cb494c72889aba4fe16050ba", client_secret="06507eb95cff48629e2db1eea2395f88")
sp = spotipy.Spotify(auth_manager=auth_manager)

In [3]:

def extraer_datos_por_genero_y_rango(genero, rango_anyo, resultados_por_anyo=200):
    albumes = defaultdict(list)
    canciones = defaultdict(list)

    for offset in range(0, 200, 50):  # Buscar hasta 200 artistas
        artistas = sp.search(q=f'genre:{genero}', type='artist', limit=50, offset=offset)
        for artista in artistas['artists']['items']:
            artista_id = artista['id']
            artista_nombre = artista['name']

            # Extraer álbumes del artista
            albumes_data = sp.artist_albums(artista_id, album_type=['album', 'single'], limit=50)
            for album in albumes_data['items']:
                try:
                    album_lanzamiento = int(album['release_date'][:4])
                    if album_lanzamiento in rango_anyo:
                        albumes[album_lanzamiento].append({
                            'artista': artista_nombre,
                            'genero': genero,
                            'tipo': album['album_type'],
                            'nombre': album['name'],
                            'año': album_lanzamiento,
                            'id': album['id']
                        })
                except (KeyError, ValueError, TypeError):
                    continue

            # Extraer top tracks del artista (sin vincular a álbumes)
            try:
                top_tracks = sp.artist_top_tracks(artista_id)
                for track in top_tracks['tracks']:
                    try:
                        track_lanzamiento = int(track['album']['release_date'][:4])
                        if track_lanzamiento in rango_anyo:
                            canciones[track_lanzamiento].append({
                                'artista': artista_nombre,
                                'genero': genero,
                                'tipo': 'cancion',
                                'nombre': track['name'],
                                'año': track_lanzamiento,
                                'id': track['id']
                            })
                    except (KeyError, ValueError, TypeError):
                        continue
            except:
                continue

            # Parar si ya tenemos suficientes resultados para todos los años
            if all(len(albumes.get(anyo, [])) >= resultados_por_anyo and len(canciones.get(anyo, [])) >= resultados_por_anyo for anyo in rango_anyo):
                break

    # Devolver resultados mezclados (o por separado si prefieres)
    return [
        {'tipo': 'album', **data} for anyo in rango_anyo for data in albumes[anyo][:resultados_por_anyo]
    ] + [
        {'tipo': 'cancion', **data} for anyo in rango_anyo for data in canciones[anyo][:resultados_por_anyo]
    ]




In [4]:
tastyinfo_2018_2022 = extraer_datos_por_genero_y_rango('rock', range(2018, 2023))

In [ ]:
type(tastyinfo_2018_2022)

list

In [ ]:
tastyinfo_2018_2022

[{'tipo': 'album',
  'artista': 'Bunbury',
  'genero': 'rock',
  'nombre': 'Canciones 1987-2017 (2018 Remaster)',
  'año': 2018,
  'id': '6z14BU1sSs5vpswjtNRz5t'},
 {'tipo': 'album',
  'artista': 'R.E.M.',
  'genero': 'rock',
  'nombre': 'R.E.M. At The BBC (Live)',
  'año': 2018,
  'id': '35B6Tl6RKNSP6fyZrzR6Qb'},
 {'tipo': 'album',
  'artista': 'Muse',
  'genero': 'rock',
  'nombre': 'Simulation Theory (Super Deluxe)',
  'año': 2018,
  'id': '5OZgDtx180ZZPMpm36J2zC'},
 {'tipo': 'album',
  'artista': 'David Bowie',
  'genero': 'rock',
  'nombre': 'Glastonbury 2000 (Live)',
  'año': 2018,
  'id': '6frjpRu2P0fdF9xSS9stjY'},
 {'tipo': 'album',
  'artista': 'David Bowie',
  'genero': 'rock',
  'nombre': "Welcome To The Blackout (Live London '78)",
  'año': 2018,
  'id': '0blyxTUATvmwqDcVM0dGt6'},
 {'tipo': 'album',
  'artista': 'Andrés Calamaro',
  'genero': 'rock',
  'nombre': 'Cargar La Suerte',
  'año': 2018,
  'id': '5y3l6sn4M3ACns2H51wGrl'},
 {'tipo': 'album',
  'artista': 'Bon Jovi',

In [ ]:
#### MUY PROBABLEMENTE ESTE PROCESO DE LIMPIEZA NO HACE FALTA ####

# Filtrar álbumes y canciones
albumes = [item for item in tastyinfo_2018_2022 if item['tipo'] in ['album', 'single']]
canciones = [item for item in tastyinfo_2018_2022 if item['tipo'] == 'cancion']

# Limpiar álbumes (opcional, si solo quieres ciertos campos)
albumes_limpios = [
    {
        'artista': album['artista'],
        'nombre': album['nombre'],
        'año': album['año'],
        'tipo': album['tipo']
    }
    for album in albumes
]

# Limpiar canciones (opcional, si solo quieres ciertos campos)
canciones_limpias = [
    {
        'artista': cancion['artista'],
        'nombre': cancion['nombre'],
        'año': cancion['año'],
        'tipo': cancion['tipo']
    }
    for cancion in canciones
]

print("Álbumes limpios:")
for album in albumes_limpios:  # Muestra solo los primeros 5 para no saturar la salida
    print(album)
    
print("\nCanciones limpias:")
print('-----------------------------------')
for cancion in canciones_limpias:  # Muestra solo los primeros 5 para no saturar la salida
    print(cancion)


Álbumes limpios:
{'artista': 'Bunbury', 'nombre': 'Canciones 1987-2017 (2018 Remaster)', 'año': 2018, 'tipo': 'album'}
{'artista': 'R.E.M.', 'nombre': 'R.E.M. At The BBC (Live)', 'año': 2018, 'tipo': 'album'}
{'artista': 'Muse', 'nombre': 'Simulation Theory (Super Deluxe)', 'año': 2018, 'tipo': 'album'}
{'artista': 'David Bowie', 'nombre': 'Glastonbury 2000 (Live)', 'año': 2018, 'tipo': 'album'}
{'artista': 'David Bowie', 'nombre': "Welcome To The Blackout (Live London '78)", 'año': 2018, 'tipo': 'album'}
{'artista': 'Andrés Calamaro', 'nombre': 'Cargar La Suerte', 'año': 2018, 'tipo': 'album'}
{'artista': 'Bon Jovi', 'nombre': 'This House Is Not For Sale (Deluxe)', 'año': 2018, 'tipo': 'album'}
{'artista': 'Bon Jovi', 'nombre': 'This House Is Not For Sale', 'año': 2018, 'tipo': 'album'}
{'artista': 'Hombres G', 'nombre': 'Huevos Revueltos (En Vivo)', 'año': 2018, 'tipo': 'album'}
{'artista': 'Fleetwood Mac', 'nombre': "50 Years - Don't Stop", 'año': 2018, 'tipo': 'album'}
{'artista': 

In [ ]:
df_2018_2022_sp = pd.DataFrame(tastyinfo_2018_2022)

        tipo               artista genero  \
0      album               Bunbury   rock   
1      album                R.E.M.   rock   
2      album                  Muse   rock   
3      album           David Bowie   rock   
4      album           David Bowie   rock   
..       ...                   ...    ...   
669  cancion              Iggy Pop   rock   
670  cancion                  LiSA   rock   
671  cancion                  LiSA   rock   
672  cancion                  LiSA   rock   
673  cancion  Amistades Peligrosas   rock   

                                        nombre   año                      id  
0          Canciones 1987-2017 (2018 Remaster)  2018  6z14BU1sSs5vpswjtNRz5t  
1                     R.E.M. At The BBC (Live)  2018  35B6Tl6RKNSP6fyZrzR6Qb  
2             Simulation Theory (Super Deluxe)  2018  5OZgDtx180ZZPMpm36J2zC  
3                      Glastonbury 2000 (Live)  2018  6frjpRu2P0fdF9xSS9stjY  
4    Welcome To The Blackout (Live London '78)  2018  0blyxTUATv

In [13]:
df_2018_2022_sp.to_csv('spotify_data.csv', index=False, encoding='utf-8')